In [2]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import re, nltk, random, os, json
import http.client, urllib.request, urllib.parse, urllib.error, base64
from nltk.corpus import stopwords
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from rank_bm25 import BM25Okapi

# keras
from keras.layers import Input, LSTM, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Embedding, Multiply, Dot, Dense, Subtract, Activation, SimpleRNN, Flatten, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np

In [57]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc):
    doc = regex.sub(' ', doc)
#     doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower()

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    result = {}
    
    
    
    for i in [5, 10 ,20]:
        metric = MSnDCG(xrelnum, grades, cutoff=i)
        result["ndcg@%d" % i] = metric.compute(labeled_ranked_list)
        
        _ranked_list = ranked_list[:i]
        result["p@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(_ranked_list)
        result["r@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(qrels)
        
    return result

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches]).lower()

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }
    
    queries = [str(i).replace("\'", "") for i in queries]

    params = urllib.parse.urlencode({})
    
    while True:
        try:
            conn = http.client.HTTPSConnection('api.msturing.org')
    #         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
            conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)
            conn.close()
            break
        except Exception as e:
    #         print(data)
            print("delay")
            time.sleep(5)
#         print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

def getTermMSvec(all_properties):
    tmp = {}
    for i in range(0, len(all_properties), 20):
        data = getVectors(all_properties[i:i+20])
        for i in data:
            tmp[i] = data[i]
    return tmp

In [16]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to", "")

df_wiki['overview'] = [preprocessingText(str(i)) for i in df_wiki['overview']]
df_wiki['headline'] = [preprocessingText(str(i)) for i in df_wiki['headline']]
df_wiki['text'] = [preprocessingText(str(i)) for i in df_wiki['text']]
df_wiki['sectionLabel'] = [preprocessingText(str(i)) for i in df_wiki['sectionLabel']]
df_wiki['title'] = [preprocessingText(str(i)) for i in df_wiki['title']]
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [13]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")
df_action = pd.read_csv("data/AKG/Test Collection/AM/akg_standard_am_verb_object_rele.csv")

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})
for c in ["query", "entityType", "action", "entity"]:
    topic[c] = topic[c].str.lower().replace("\'", "")
    
df = df.merge(topic, how="inner", on="query_id")
# df['query'] = df[['action', 'entity', 'entityType']].astype(str).apply(' '.join, axis=1)

In [17]:
dfw = df_wiki[df_wiki.title.str.contains("|".join(df.entity.unique().tolist()))]

In [23]:
entityInWiki = []
for e in df.entity.unique():
    tmp = dfw[dfw.title.str.contains(e)]
    if len(tmp) > 0:
        entityInWiki.append(e)

In [104]:
node2MSvec = {}
total = 0
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    query = row['action'] + " " + row['entity']
    node2MSvec[query] = getVectors([query])[query]
    if row['query_id'] in qid2MSvec:
        continue
    if row['entity'] not in entityInWiki:
        continue   
#     titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
#     titles = [i[1:] for i in titles]
#     for i in range(0, len(titles), 20):
#         data = getVectors(titles[i:i+20])
#         for j in data:
#             node2MSvec[j] = data[j]                    
#     data = getTermMSvec([query] + titles)
#     rank = {}
#     for i in data:
#         if i == query:
#             continue
#         rank[i] = cosine_similarity([data[query]], [data[i]])[0][0]
#     best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
#     sentences = []
#     tokens = " ".join(dfw[dfw.title == best_title].text.tolist()).split()
#     for i in range(0, len(tokens), 10):
#         sentences.append(" ".join(tokens[i:i+10]))
# #     print(sentences)
#     data = getTermMSvec([query] + sentences)
#     qid2MSvec[row['query_id']] = list(data.values())


KeyError: "hear baby 's heartbeat heart rate monitor"

In [91]:
import pickle
f = open("wiki2vec.pkl","wb")
pickle.dump(node2MSvec,f)
f.close()

In [93]:
a = pickle.load( open("wiki2vec.pkl", "rb"))